# Create region to average RAP analysis over

#### Need to change location of RAP analysis file output for external use

# Great circle distance calculation function

In [1]:
def rap13_great_circle(rap_ds, lat, lon, radius):

    # Convert RAP coordinate units and input latitude and longitude to radians
    rap_lat = rap_ds.latitude * (np.pi/180)
    rap_lon = rap_ds.longitude * (np.pi/180)
            
    lat, lon = map(radians, [lat, lon])

    # Create variable for number of grid points within 60km of input latitude and longitude
    num_points = 0
    
    # Create arrays for grid points within 60km of input latitude and longitude
    grid_points_x = []
    grid_points_y = []
    
    # Create array so distance calculation works over entirety of RAP analysis domain
    distance = np.zeros((337, 451))
        
    # Calculate great-circle distance equation in km
    for i in range(len(rap_ds.latitude)):
        for j in range(len(rap_ds.latitude[i])):
            distance[i,j] = 6371 * (acos(sin(lat) * sin(rap_lat[i,j]) + cos(lat) * cos(rap_lat[i,j]) * cos(lon - rap_lon[i,j])))
            if distance[i,j] <= radius:
                grid_points_y.append(i)
                grid_points_x.append(j)
                num_points += 1
    
    # Return desired outputs
    return grid_points_x, grid_points_y, num_points

# Unit test to determine if great circle function works

In [2]:
def great_circle_test(test_rap_ds):
    '''Test that great circle function generates the correct number of points'''
    
    # Set lat, lon, radius, and constants (do not edit)
    test_num_points = 0
    test_lat = 35
    test_lon = -100
    test_radius = 10
    
    # Convert RAP coordinate units and input latitude and longitude to radians
    test_rap_lat = test_rap_ds.latitude * (np.pi/180)
    test_rap_lon = test_rap_ds.longitude * (np.pi/180)
    test_lat, test_lon = map(radians, [test_lat, test_lon])
    
    # Create array so distance calculation works over entirety of RAP analysis domain
    test_distance = np.zeros((337, 451))
    
    # Calculate great-circle distance equation in km
    for i in range(len(test_rap_ds.latitude)):
        for j in range(len(test_rap_ds.latitude[i])):
            test_distance[i,j] = 6371 * (acos(sin(test_lat) * sin(test_rap_lat[i,j]) + cos(test_lat) * cos(test_rap_lat[i,j]) * cos(test_lon - test_rap_lon[i,j])))
            if test_distance[i,j] <= test_radius:
                test_num_points += 1
                
    # Output error if calculation produces incorrect output
    if (test_num_points == 2):
        pass
    else:
        sys.exit('❌ Error. Calculation not valid. Possible error with RAP grid.')

# Output RAP gird point over user defined radius centered around latitude and longitude

In [3]:
import numpy as np
import xarray as xr
from math import radians, cos, sin, acos, sqrt
import urllib.request
import sys

# Import warnings package to prevent non-critical warnings from displaying
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Read in RAP analysis dataset
urllib.request.urlretrieve(f"https://www.ncei.noaa.gov/thredds/fileServer/model-rap130anl-old/201407/20140703/rap_130_20140703_2000_000.grb2", 
                                       f"/home/scratch/nsonntag/eae598/rap_data/rap_130_20140703_2000_000.grb2")

ds = xr.open_dataset("/home/scratch/nsonntag/eae598/rap_data/rap_130_20140703_2000_000.grb2", filter_by_keys=
                    {'typeOfLevel': 'heightAboveGround', 'level': 2})

# Unit test to make sure RAP analysis grid is not 
great_circle_test(ds)

# Insert latitude and longitude of desired point
lat, lon = 34.2727, -77.9455

# Run great-circle distance function
grid_x, grid_y, num_points = rap13_great_circle(ds, lat, lon, radius=60)

# Print outputs
print(f"The number of points within the distance of the geographic input in RAP analysis is {num_points}.")
print(f"Point latitude values array (grid_y): {grid_y}")
print(f"Point longitude value array (grid_x): {grid_x}")

The number of points within the distance of the geographic input in RAP analysis is 65.
Point latitude values array (grid_y): [123, 123, 123, 124, 124, 124, 124, 124, 124, 124, 125, 125, 125, 125, 125, 125, 125, 126, 126, 126, 126, 126, 126, 126, 126, 126, 127, 127, 127, 127, 127, 127, 127, 127, 127, 128, 128, 128, 128, 128, 128, 128, 128, 128, 129, 129, 129, 129, 129, 129, 129, 129, 129, 130, 130, 130, 130, 130, 130, 130, 131, 131, 131, 131, 131]
Point longitude value array (grid_x): [362, 363, 364, 360, 361, 362, 363, 364, 365, 366, 360, 361, 362, 363, 364, 365, 366, 359, 360, 361, 362, 363, 364, 365, 366, 367, 359, 360, 361, 362, 363, 364, 365, 366, 367, 359, 360, 361, 362, 363, 364, 365, 366, 367, 359, 360, 361, 362, 363, 364, 365, 366, 367, 360, 361, 362, 363, 364, 365, 366, 361, 362, 363, 364, 365]
